In [1]:
import json
import re
from time import time
import numpy as np
import pandas as pd
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
# Get the evalation spec/sheets
file_loc = "./2023-09-08 - Data Ateliers.xlsx"
# --
sheet1 = "Experiences - Réponses Similair"
exp1_df = pd.read_excel(file_loc, sheet_name=sheet1)
exp1_df.dropna(how="all", inplace=True)
exp1_df = exp1_df.astype(str)

sheet2 = "Experiences - service-public.fr"
exp2_df = pd.read_excel(file_loc, sheet_name=sheet2)
exp2_df.dropna(how="all", inplace=True)
exp2_df = exp2_df.astype(str)


sheet3 = "Experiences - Modèle 13B"
exp3_df = pd.read_excel(file_loc, sheet_name=sheet3)
exp3_df.dropna(how="all", inplace=True)
exp3_df = exp3_df.astype(str)


# Get the experiences
with open("../_data/export-expa-c-riences.json") as f:
    documents = json.load(f)

documents = {doc["id_experience"]: doc for doc in documents}

In [3]:
# EXP 1
# - from the given experience ID
# - search for more similare experience, return experience (should have a response present!)

url = "https://gpt.datascience.etalab.studio/api/search/experiences"
headers = {
    "Content-Type": "application/json"
}

for i, line in exp1_df.iterrows():
    id_experience = int(line["ID experience"])
    exp = documents[id_experience]
    #print("source:", id_experience)

    # Bm25
    query = {"q": exp["description"], "n": 5, "similarity": "bm25"}
    res = requests.post(url, headers=headers, data=json.dumps(query), verify=False).json()
    best_bm25 = None
    for hit in res:
        if id_experience == hit["id_experience"]:
            continue
        if not hit["reponse_structure_1"]:
            continue
        best_bm25 = hit
        break
        
    if not best_bm25:
        print("WARNING: no bm25 best found !")
        continue

    # e5
    query = {"q": exp["description"], "n": 5, "similarity": "e5"}
    res = requests.post(url, headers=headers, data=json.dumps(query), verify=False).json()
    best_e5 = None
    for hit in res:
        if id_experience == hit["id_experience"]:
            continue
        if not hit["reponse_structure_1"]:
            continue
        best_e5 = hit
        break
        
    if not best_e5:
        print("WARNING: no e5 best found !")
        continue

    # Fill the sheet
    exp1_df.at[i, "Titre"] =  exp["titre"]
    exp1_df.at[i, "Description"] =  exp["description"]
    exp1_df.at[i, "Intitulé Typologie 1"] =  exp["intitule_typologie_1"]
    exp1_df.at[i, "Intitulé structure 1"] =  exp["intitule_structure_1"]
    exp1_df.at[i, "Réponse similaire Algo Lexical"] =  best_bm25["description"]
    exp1_df.at[i, "Réponse similaire Algo Sémantique"] =  best_e5["description"]
    exp1_df.at[i, "Réponse structure 1"] =  exp["reponse_structure_1"]

exp1_df

,ID experience,Titre,Description,Intitulé Typologie 1,Intitulé structure 1,Réponse similaire Algo Lexical,Réponse similaire Algo Sémantique,Réponse structure 1
0,3788186,Impossibilité de me connecter sur mon compte A...,J'ai souhaiter me connecter sur mon compte Amé...,CNAM,CNAM,Depuis une semaine je ne peux plus me connecte...,Après un essai non concluant fait sur interne...,"Bonjour Doudou,\n\nNous sommes navrés des désa..."
1,3783211,Ignorance de ma demande d’invalidité,Je suis en mi-temps thérapeutique depuis le 7 ...,CNAM,CNAM,"Bonjour, j'ai déposé ma demande de retraite en...",Après un essai non concluant fait sur interne...,"Bonjour Amine,\n\nNous vous remercions d'avoir..."
2,3771413,mon espace sante,Il est impossible en tant que professionnel de...,CNAM,CNAM,"Bonjour,\nJe me suis inscrit sans problème et ...","Pour une demande d'attestation, ma 1ère répons...","BonjourPHCH,\n\nNous vous remercions pour votr..."
3,3729951,Fiasco total,Cela fait plus d’an que j’ai déposé ma demande...,Préfecture,Préfecture des Hauts-de-Seine,J’ai déposé une demande de naturalisation à la...,"Madame, Monsieur,\n\nVeuillez accepter d'avanc...","Bonjour,&nbsp;\n\nLe bureau des naturalisation..."
4,3727833,Déjà 6 mois d’attente sans suite,J’ai été informé avant de faire ma demande que...,Préfecture,Préfecture de l'Essonne,Bonjour. \n\nJe suis réfugié politique biéloru...,Après un essai non concluant fait sur interne...,"Bonjour Exaucé,\n\nMerci d'avoir partagé votre..."
5,3690465,Impossibilité d'aboutir : demande en ligne act...,"Bonjour,\nMa fille est née à l'étranger. Je ve...",Ministère de l’Europe et des Affaires étrangèr...,Direction des Français à l'étranger et de l'ad...,Je navigue depuis hier de France connect à ser...,"Madame, Monsieur,\n\nVeuillez accepter d'avanc...","Bonjour dro58,\n\nNous vous remercions d'avoir..."
6,3255291,DCEM demande en ligne,Le système ne fonctionne pas. A la premiere èt...,Ministère de l’Europe et des Affaires étrangèr...,Hors-structures,"Par le site ANTS, un de mes proches, de nation...","Le : "" 0 "" (zéro) n'est pas nul dans certaines...",Le service public concerné par votre témoignag...
7,3800342,Passesport Mineur,"Bonjour,\n\nAyant fait la demande de passeport...",ANTS,ANTS,"Bonjour,\nJ'ai effectué ma pré-demande en lign...","Madame, Monsieur,\n\nVeuillez accepter d'avanc...",None
8,3799111,carte grise,"Suite au décès de mon père, j'attends la carte...",ANTS,ANTS,Suite à une demande d’immatriculation d’un véh...,"Bonsoir , ce formulaire est explicite mais dif...",None
9,3798676,Acces a ants,je souhaite prendre rendez vous pour changer m...,ANTS,ANTS,J'ai besoin de récupérer une attestation de pa...,Je désirais savoir si je pouvais effectuer ma ...,None


In [4]:
# EXP 2
# - from the given experience  ID
# - search for the more similare sheets
# EXP 1
# - from the given experience ID
# - search for more similare experience, return experience (should have a response present!)

url = "https://gpt.datascience.etalab.studio/api/search/sheets"
headers = {
    "Content-Type": "application/json"
}

for i, line in exp2_df.iterrows():
    id_experience = int(line["ID experience"])
    exp = documents[id_experience]
    #print("source:", id_experience)

    # Bm25
    query = {"q": exp["description"], "n": 2, "similarity": "bm25"}
    res = requests.post(url, headers=headers, data=json.dumps(query), verify=False).json()
    best_bm25 = None
    for hit in res:
        best_bm25 = hit
        break
        
    if not best_bm25:
        print("WARNING: no bm25 best found !")
        continue

    # e5
    query = {"q": exp["description"], "n": 2, "similarity": "e5"}
    res = requests.post(url, headers=headers, data=json.dumps(query), verify=False).json()
    best_e5 = None
    for hit in res:
        best_e5 = hit
        break
        
    if not best_e5:
        print("WARNING: no e5 best found !")
        continue

    # Fill the sheet
    exp2_df.at[i, "Titre"] =  exp["titre"]
    exp2_df.at[i, "Description"] =  exp["description"]
    exp2_df.at[i, "Intitulé Typologie 1"] =  exp["intitule_typologie_1"]
    exp2_df.at[i, "Intitulé structure 1"] =  exp["intitule_structure_1"]
    exp2_df.at[i, "Fiche Algo Lexical"] =  best_bm25["url"]
    exp2_df.at[i, "Fiche Algo Sémantique"] =  best_e5["url"]

exp2_df

,ID experience,Titre,Description,Intitulé Typologie 1,Intitulé structure 1,Fiche Algo Lexical,Fiche Algo Sémantique,Unnamed: 7
0,3783539,Il me manque des indemnités journalières,Il me manque des indemnités journalières après...,CNAM,CNAM,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan
1,3783244,Demande de renouvellement Complémentaire santé...,Je suis bénéficiaire de la Complémentaire sant...,CNAM,CNAM,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/professionnels-e...,nan
2,3655311,Titre de séjour,"Bonjour, \nAprès obtention de mon master, mon ...",Préfecture,Préfecture du Val-d'Oise,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan
3,3707569,Demande de naturalisation,Bonjour. \nIl est parfois ahurissant de voir l...,Préfecture,Préfecture de la Seine-Maritime,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan
4,3226615,Radiation des listes consulaires,J'essaie de me radier en ligne des listes cons...,Ministère de l’Europe et des Affaires étrangèr...,Direction des Français à l'étranger et de l'ad...,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/professionnels-e...,nan
5,3585707,Délais d'instruction beaucoup plus longs qu'avant,J'ai soumis ma demande de naturalisation par d...,Ministère de l’Europe et des Affaires étrangèr...,Préfecture des Hauts-de-Seine,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/professionnels-e...,nan
6,3798983,Lenteur de traitement Immatriculation Véhicule...,Après m'avoir annoncé (et pas respecté) 10 jou...,ANTS,ANTS,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan
7,3790069,MSA Marne non payement des indemnités journali...,En mi-temps thérapeutique depuis février j'enc...,MSA,MSA Marne Ardennes Meuse,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan
8,3779948,pension retraite non versée,"Bonjour,\nMalgré de multiples relances ( courr...",MSA,MSA Ile de France,https://www.service-public.fr/particuliers/vos...,https://www.service-public.fr/particuliers/vos...,nan


In [5]:
# EXP 3
# - from the given experience ID
# - Generatye a response with the 13b fabrique model

# Fill the sheet
# - Id, Réponse structure 1, Réponse Modèle 13B

In [6]:
# Save results

# Create an ExcelWriter object
writer = pd.ExcelWriter('Evaluation-atelier-dinum-1.xlsx')

# Write each DataFrame to a separate sheet in the Excel file
exp1_df.to_excel(writer, sheet_name=sheet1, index=False)
exp2_df.to_excel(writer, sheet_name=sheet2, index=False)
exp3_df.to_excel(writer, sheet_name=sheet3, index=False)

# Save the Excel file
writer.save()